# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [10]:
# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"

# assign the /status endpoint to another variable called status_url (1 line)
endpoint = "/status" 

# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f"{root_url}{endpoint}")

# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print(req.text)
else:
    print(req.status_code)


"Alive"


### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [11]:
# Set the countries_url variable (1 line)
endpoint = "/countries"

# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f"{root_url}{endpoint}")

# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()
# display the request's status code and the countries variable (1 line)
print(req.status_code)
print(countries)

403
{'message': 'The cookie is missing'}


### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [12]:
# Set the cookie_url variable (1 line)
cookie_url = f"{root_url}/cookie"
# Query the enpoint, set the cookies variable and display it (2 lines)
req = requests.get(cookie_url)
cookies = req.cookies.get_dict()
print(cookies)

{'user_cookie': 'f5a24e6c-4b4e-4aa6-b0d2-87765a5ed777'}


Try to query the countries endpoint using the cookie, save the output and print it.

In [14]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
endpoint = "/countries"
req = requests.get(f"{root_url}{endpoint}",cookies=cookies)
countries = req.json()
# display the countries variable (1 line)
print(countries)

['be', 'us', 'ma', 'fr', 'ru']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [31]:
# Set the leaders_url variable (1 line)
leaders_url = f"{root_url}/leaders"
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
req = requests.get(cookie_url)
cookies = req.cookies.get_dict()
req = requests.get(leaders_url,cookies=cookies)
leaders = req.json()
# display the leaders variable (1 line)
print(leaders)

{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [ ]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
req = requests.get(f"{root_url}/check")
if req.status_code == 403:
    req = requests.get(cookie_url)
    cookies = req.cookies.get_dict()
else:
    print("Cookies is active")
req = requests.get(f"{root_url}/countries",cookies=cookies)
countries = req.json()
req = requests.get(leaders_url,params={"country": countries[0]},cookies=cookies)
# assign the output to the leaders variable (1 line)
leaders = req.json()
# display the leaders variable (1 line)
print(leaders)

### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [ ]:
# 4 lines
leaders_per_country = {}
for country in countries:
    req = requests.get(leaders_url,params={"country":country},cookies=cookies)
    leaders_per_country[country] = req.json()

print(leaders_per_country)

In [ ]:
# or 1 line
leaders_per_country = {country:requests.get(leaders_url,params={"country":country},cookies=cookies).json() for country in countries}

print(leaders_per_country)

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [7]:
# < 15 lines
import requests
def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    # Set the cookies variable
    cookie_url = f"{root_url}/cookie"
    req = requests.get(cookie_url)
    cookies = req.cookies.get_dict()
    # Set the endpoint /countries and keep in a variable countries
    req = requests.get(f"{root_url}/countries",cookies=cookies)
    countries = req.json()
    # Make a dictionary named leaders_per_country and loop through the countries variable and insert them inside the disctionary
    leaders_url = f"{root_url}/leaders"
    leaders_per_country = {}
    leaders_per_country = {country:requests.get(leaders_url,params={"country":country},cookies=cookies).json() for country in countries}
    # display the leaders variable (1 line)
    return leaders_per_country
    

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [ ]:
# 2 lines
leaders_per_country = get_leaders()
print(leaders_per_country)

## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [ ]:
# 3 lines
headers = {
    "User-Agent": "Wikipedia-scraper/1.0 (md.maheen.billah.97@gmail.com)"
}
wiki_url = leaders_per_country["us"][0]["wikipedia_url"]
req = requests.get(wiki_url,headers=headers)
print(req.text)

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [ ]:
# 3 lines
from bs4 import BeautifulSoup
headers = {
    "User-Agent": "Wikipedia-scraper/1.0 (md.maheen.billah.97@gmail.com)"
}
wiki_url = leaders_per_country["us"][0]["wikipedia_url"]
req = requests.get(wiki_url,headers=headers)
soup = BeautifulSoup(req.content, "html")
pretty_html = soup.prettify()
txt = soup.get_text()
print(txt)

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [36]:
# 2 lines
paragraphs = soup.find_all('p')


If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

In [ ]:
# <10 lines
first_paragraph = None
for paragraph in paragraphs:
    text = paragraph.get_text().strip() #stripping white space   
    if not text:
        continue
    # if len(text) < 50:  # Adjust this threshold as needed
    #     continue
    else:  # we are hoping the length to be atleast more than 10
        first_paragraph = text
        break

print(first_paragraph)

At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [70]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    # print(wikipedia_url) # keep this for the rest of the notebook
    headers = {
    "User-Agent": "Wikipedia-scraper/1.0 (md.maheen.billah.97@gmail.com)"
    }
    req = requests.get(wikipedia_url,headers=headers)
    soup = BeautifulSoup(req.content, "html")
    paragraphs = soup.find_all('p')
    first_paragraph = None
    for paragraph in paragraphs:
        text = paragraph.get_text().strip() #stripping white space   
        if not text:
            continue
        # if len(text) < 50:  # Adjust this threshold as needed
        #     continue
        else:  # we are hoping the length to be atleast more than 10
            first_paragraph = text
            break
    return first_paragraph

In [71]:
# Test: 3 lines
p1 = get_first_paragraph(leaders_per_country["us"][0]["wikipedia_url"])


### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [ ]:
# 3 lines
# Test: 3 lines
import re
        # Remove pronunciation guides with ⓘ
p1 = re.sub(r'\([^)]*ⓘ[^)]*\)', '', p1)
        # Remove extra spaces
p1 = re.sub(r'\s+', ' ', p1).strip()
print(p1)


Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [ ]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    # print(wikipedia_url) # keep this for the rest of the notebook
    headers = {
    "User-Agent": "Wikipedia-scraper/1.0 (md.maheen.billah.97@gmail.com)"
    }
    req = requests.get(wikipedia_url,headers=headers)
    soup = BeautifulSoup(req.content, "html.parser")
    paragraphs = soup.find_all('p')
    first_paragraph = None
    for paragraph in paragraphs:
        text = paragraph.get_text().strip() #stripping white space   
        if not text:
            continue
        if len(text) < 20:  # Adjust this threshold as needed
            continue
        else:  # we are hoping the length to be atleast more than 10
            text = re.sub(r'\([^)]*ⓘ[^)]*\)', '', text)
            text = re.sub(r'\s+', ' ', text).strip()
            first_paragraph = text
            break
    return first_paragraph
get_first_paragraph(leaders_per_country["be"][0]["wikipedia_url"])

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [130]:
# < 20 lines
def get_first_paragraph(wikipedia_url):
    # print(wikipedia_url) # keep this for the rest of the notebook
    headers = {
    "User-Agent": "Wikipedia-scraper/1.0 (md.maheen.billah.97@gmail.com)"
    }
    req = requests.get(wikipedia_url,headers=headers)
    soup = BeautifulSoup(req.content, "html.parser")
    paragraphs = soup.find_all('p')
    first_paragraph = None
    for paragraph in paragraphs:
        text = paragraph.get_text().strip() #stripping white space   
        if not text:
            continue
        if paragraph.find_parent('div', class_='hatnote') or \
           paragraph.find_parent('div', class_='dablink') or \
           paragraph.find_parent('div', class_='homonymie') or \
           paragraph.find_parent('div', class_='bandeau') or \
           paragraph.find_parent('div', class_='mw-flagged') or \
           paragraph.find_parent('div', class_='mw-checked') or \
           paragraph.find_parent('div', class_='verified') or \
            paragraph.find_parent('div', class_='mw-verified') or \
            paragraph.find_parent('div', class_='metadata'): 
            continue
        if len(text) < 20:  # Adjust this threshold as needed
            continue
        if "متحقق منها" in text or "مفحوصة" in text:  
            continue
        else:  # we are hoping the length to be atleast more than 20
            text = re.sub(r'\[[^\]]*\]', '', text)
            text = re.sub(r'\(\s*[^)]*[ɑɛœɔʁɡʒʃɥɑ̃ɛ̃ɔ̃][^)]*\)', '', text)
            text = re.sub(r'Écouterⓘ', '', text)
            text = re.sub(r'\[\d+\]', '', text)
            text = re.sub(r'\[[a-z]\]', '', text) 
            text = re.sub(r'\([^)]*ⓘ[^)]*\)', '', text)
            text = re.sub(r'\([^)]*[ɑɛœɔʁɡʒʃ][^)]*\)', '', text)
            text = re.sub(r'\(\s*\)', '', text)
            text = re.sub(r'\s+', ' ', text).strip()
            first_paragraph = text
            break
    return first_paragraph

## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [131]:
# < 20 lines
import time
def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    # Set the cookies variable
    cookie_url = f"{root_url}/cookie"
    req = requests.get(cookie_url)
    cookies = req.cookies.get_dict()
    # Set the endpoint /countries and keep in a variable countries
    req = requests.get(f"{root_url}/countries",cookies=cookies)
    countries = req.json()
    # Make a dictionary named leaders_per_country and loop through the countries variable and insert them inside the disctionary
    leaders_url = f"{root_url}/leaders"
    leaders_per_country = {}
    leaders_per_country = {country:requests.get(leaders_url,params={"country":country},cookies=cookies).json() for country in countries}
    for country in leaders_per_country:
        print(f"Country: {country.upper()}\n")
        for leader in leaders_per_country[country]:
            wikipedia_url = leader["wikipedia_url"]
            fname = leader["first_name"]
            lname = leader["last_name"]
            if  not lname or lname == "None":
                print(f"  Leader: {fname}")
            else:
                print(f"  Leader: {fname} {lname}")
            print(f"  URL: {wikipedia_url}")
            paragraph = get_first_paragraph(wikipedia_url)
            print(f"  Bio: {paragraph}\n")
            time.sleep(2)
    # display the leaders variable (1 line)
    return leaders_per_country

In [ ]:
# Check the output of your function (2 lines)
get_leaders()

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [128]:
# < 25 lines
import time
def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    # Set the cookies variable
    cookie_url = f"{root_url}/cookie"
    req = requests.get(cookie_url)
    cookies = req.cookies.get_dict()
    # Set the endpoint /countries and keep in a variable countries
    req = requests.get(f"{root_url}/countries",cookies=cookies)
    countries = req.json()
    # Make a dictionary named leaders_per_country and loop through the countries variable and insert them inside the disctionary
    leaders_url = f"{root_url}/leaders"
    leaders_per_country = {}
    leaders_per_country = {country:requests.get(leaders_url,params={"country":country},cookies=cookies).json() for country in countries}
    for country in leaders_per_country:
        print(f"Country: {country.upper()}\n")
        for leader in leaders_per_country[country]:
            wikipedia_url = leader["wikipedia_url"]
            fname = leader["first_name"]
            lname = leader["last_name"]
            if  not lname or lname == "None":
                print(f"  Leader: {fname}")
            else:
                print(f"  Leader: {fname} {lname}")
            print(f"  URL: {wikipedia_url}")
            paragraph = get_first_paragraph(wikipedia_url)
            print(f"  Bio: {paragraph}\n")
            time.sleep(2)
    # display the leaders variable (1 line)
    return leaders_per_country


Check the output of your function again.

In [ ]:
# Check the output of your function (1 line)
get_leaders()

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [133]:
# < 20 lines
from requests import Session
def get_first_paragraph(wikipedia_url, session: Session):
    # print(wikipedia_url) # keep this for the rest of the notebook
    headers = {
    "User-Agent": "Wikipedia-scraper/1.0 (md.maheen.billah.97@gmail.com)"
    }
    req = session.get(wikipedia_url,headers=headers)
    soup = BeautifulSoup(req.content, "html.parser")
    paragraphs = soup.find_all('p')
    first_paragraph = None
    for paragraph in paragraphs:
        text = paragraph.get_text().strip() #stripping white space   
        if not text:
            continue
        if paragraph.find_parent('div', class_='hatnote') or \
           paragraph.find_parent('div', class_='dablink') or \
           paragraph.find_parent('div', class_='homonymie') or \
           paragraph.find_parent('div', class_='bandeau') or \
           paragraph.find_parent('div', class_='mw-flagged') or \
           paragraph.find_parent('div', class_='mw-checked') or \
           paragraph.find_parent('div', class_='verified') or \
            paragraph.find_parent('div', class_='mw-verified') or \
            paragraph.find_parent('div', class_='metadata'): 
            continue
        if len(text) < 20:  # Adjust this threshold as needed
            continue
        if "متحقق منها" in text or "مفحوصة" in text:  
            continue
        else:  # we are hoping the length to be atleast more than 20
            text = re.sub(r'\[[^\]]*\]', '', text)
            text = re.sub(r'\(\s*[^)]*[ɑɛœɔʁɡʒʃɥɑ̃ɛ̃ɔ̃][^)]*\)', '', text)
            text = re.sub(r'Écouterⓘ', '', text)
            text = re.sub(r'\[\d+\]', '', text)
            text = re.sub(r'\[[a-z]\]', '', text) 
            text = re.sub(r'\([^)]*ⓘ[^)]*\)', '', text)
            text = re.sub(r'\([^)]*[ɑɛœɔʁɡʒʃ][^)]*\)', '', text)
            text = re.sub(r'\(\s*\)', '', text)
            text = re.sub(r'\s+', ' ', text).strip()
            first_paragraph = text
            break
    return first_paragraph

Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [140]:
# <25 lines
def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    # Set the cookies variable
    cookie_url = f"{root_url}/cookie"
    req = requests.get(cookie_url)
    cookies = req.cookies.get_dict()
    # Set the endpoint /countries and keep in a variable countries
    req = requests.get(f"{root_url}/countries",cookies=cookies)
    countries = req.json()
    #wikipedia sessions
    wikipedia_session = requests.Session()
    wikipedia_session.headers.update({
        "User-Agent": "Wikipedia-scraper/1.0 (md.maheen.billah.97@gmail.com)"
    })
    # Make a dictionary named leaders_per_country and loop through the countries variable and insert them inside the disctionary
    leaders_url = f"{root_url}/leaders"
    leaders_per_country = {}
    leaders_per_country = {country:requests.get(leaders_url,params={"country":country},cookies=cookies).json() for country in countries}
    for country in leaders_per_country:
        print(f"Country: {country.upper()}\n")
        for leader in leaders_per_country[country]:
            wikipedia_url = leader["wikipedia_url"]
            fname = leader["first_name"]
            lname = leader["last_name"]
            if  not lname or lname == "None":
                print(f"  Leader: {fname}")
            else:
                print(f"  Leader: {fname} {lname}")
            print(f"  URL: {wikipedia_url}")
            paragraph = get_first_paragraph(wikipedia_url,wikipedia_session)
            leader['bio'] = paragraph
            print(f"  Bio: {paragraph}\n")
            time.sleep(0.5)
    # display the leaders variable (1 line)
    return leaders_per_country

Test your new functions.



In [ ]:
leaders_per_country = get_leaders()
print(leaders_per_country)

## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [ ]:
import json
with open('leaders.json', 'w', encoding='utf-8') as f:
    json.dump(leaders_per_country, f, indent=2, ensure_ascii=False)
    print("Data saved to leaders_per_country.json")

Data saved to leaders_per_country.json


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [ ]:
# 3 lines
with open('leaders.json', 'r', encoding='utf-8') as f:
    loaded_data = json.load(f)

if leaders_per_country == loaded_data:
    print(" Data successfully matched perfectly!")
else:
    print("Data mismatch!")

 Data successfully matched perfectly!


Make a function `save(leaders_per_country)` to call this code easily.

In [ ]:
# 3 lines
def save(leaders_per_country):
    with open('leaders.json', 'w', encoding='utf-8') as f:
        json.dump(leaders_per_country, f, indent=2, ensure_ascii=False)
        print("Data saved to leaders_per_country.json")
    with open('leaders.json', 'r', encoding='utf-8') as f:
        loaded_data = json.load(f)
    if leaders_per_country == loaded_data:
        print("Data successfully matched perfectly!")
    else:
        print("Data mismatch!")



Data saved to leaders_per_country.json
Data successfully matched perfectly!


In [151]:
# Call the function (1 line)
save(leaders_per_country)

Data saved to leaders_per_country.json
Data successfully matched perfectly!


## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!